In [2]:
from transformers import pipeline
import pandas as pd

/opt/anaconda3/envs/hackathon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data=pd.read_csv('data/aweekstocks.csv')

In [4]:
data.columns

Index(['start_date', 'end_date', 'company', 'url', 'url_mobile', 'title',
       'seendate', 'socialimage', 'domain', 'language', 'sourcecountry'],
      dtype='object')

In [5]:
classifier = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")

In [7]:
sentiment=classifier(data['title'].tolist())

In [8]:
data['sentiment']=sentiment

In [15]:
data.to_csv('data/sentiment_1week_nifty_50.csv', index=False)

In [4]:
data=pd.read_csv('data/sentiment_1week_nifty_50.csv')

In [6]:
df=pd.read_csv('data/ind_nifty50list.csv')

In [ ]:
df1=pd.DataFrame()
df1['company']=df['Company Name']
df1['Symbol']=df['Symbol']

In [15]:
df_new=pd.merge(data, df1, on='company', how='left')

In [16]:
df_new.shape

(217, 15)

In [162]:
def f1_sentiment(example):
    try:
        return (eval(example)['label'], eval(example)['score'])
    except:
        return ('failed', 0)

In [163]:
df['clean_sentiment']=df['sentiment'].apply(f1_sentiment)

In [166]:
df['label']=df['clean_sentiment'].apply(lambda x: x[0])
df['score']=df['clean_sentiment'].apply(lambda x: x[1])

In [174]:
df['adjusted_score']=df[['label', 'score']].apply(lambda x: -1*x[1] if x[0]=='NEGATIVE' else x[1], axis=1)

/var/folders/6f/w7jy48mn1d768r1pjtl55ck80000gn/T/ipykernel_21007/687308986.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['adjusted_score']=df[['label', 'score']].apply(lambda x: -1*x[1] if x[0]=='NEGATIVE' else x[1], axis=1)


In [177]:
df_new=df.groupby(['start_date'])['adjusted_score'].agg(['count', 'mean']).reset_index()

In [ ]:
df_new.to_csv('sentiment_1week_nifty_50_v2.csv', index=False)